In [ ]:
pip install s3fs

In [ ]:
pip install kafka-python

In [ ]:
from kafka import KafkaConsumer
from json import loads
import json
#from s3fs import S3FileSystem

In [ ]:
Consumer = KafkaConsumer(
    'test',
    bootstrap_servers=['54.83.103.179:9092'],
    value_deserializer=lambda x: loads(x.decode('utf-8')))

In [ ]:
for c in Consumer:
    print(c.value)

In [ ]:
from kafka import KafkaConsumer
from json import loads, dumps
import boto3
import pandas as pd
from datetime import datetime

# --- AWS S3 setup ---
s3 = boto3.client('s3')
BUCKET_NAME = 'kafka-crypto-mitul'     # ⬅️ change this to your actual bucket name
S3_PATH_PREFIX = 'kafka-data/'       # optional folder path in bucket

# --- Kafka setup ---
consumer = KafkaConsumer(
    'testing',
    bootstrap_servers=['54.83.103.179:9092'],
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)

print("✅ Connected to Kafka. Listening and writing to S3...\n")

# --- Collect messages into a small batch ---
batch = []
BATCH_SIZE = 100   # number of messages before upload

for msg in consumer:
    data = msg.value
    batch.append(data)
    print(f"🪙 {data['symbol']} | ${data['priceUsd']}")

    # When batch reaches threshold → upload to S3
    if len(batch) >= BATCH_SIZE:
        # Create DataFrame
        df = pd.DataFrame(batch)

        # Create unique file name
        file_name = f"coincap_batch_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}.csv"

        # Save locally first
        df.to_csv(file_name, index=False)
        print(f"💾 Batch saved locally: {file_name}")

        # Upload to S3
        s3.upload_file(file_name, BUCKET_NAME, f"{S3_PATH_PREFIX}{file_name}")
        print(f"☁️ Uploaded {file_name} → s3://{BUCKET_NAME}/{S3_PATH_PREFIX}{file_name}")

        # Clear the batch for the next upload
        batch = []


In [1]:
from kafka import KafkaConsumer
from json import loads, dumps
import boto3
from datetime import datetime
import json

# --- AWS S3 setup ---
s3 = boto3.client('s3')
BUCKET_NAME = 'kafka-crypto-mitul'      # your bucket
S3_PATH_PREFIX = 'kafka-data/'          # same prefix

# --- Kafka setup ---
consumer = KafkaConsumer(
    'testing',
    bootstrap_servers=['54.83.103.179:9092'],
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)

print("✅ Connected to Kafka. Listening and writing JSON to S3...\n")

# --- Batch setup ---
batch = []
BATCH_SIZE = 100   # upload every 100 messages

def upload_json_batch(records):
    """Upload a JSON batch directly to S3 without saving locally."""
    if not records:
        return

    # Timestamp-based file name
    file_name = f"coincap_batch_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}.json"

    # Full S3 key path
    s3_key = f"{S3_PATH_PREFIX}{file_name}"

    # Convert Python list → JSON string
    json_body = json.dumps(records)

    try:
        s3.put_object(
            Bucket=BUCKET_NAME,
            Key=s3_key,
            Body=json_body
        )
        print(f"📤 Uploaded {len(records)} records → s3://{BUCKET_NAME}/{s3_key}\n")

    except Exception as e:
        print(f"❌ Error uploading batch: {e}")

# --- Consume messages forever ---
for msg in consumer:
    data = msg.value
    batch.append(data)

    print(f"🪙 {data['symbol']} | ${data['priceUsd']}")

    # Upload batch once full
    if len(batch) >= BATCH_SIZE:
        upload_json_batch(batch)
        batch = []  # reset for next batch

✅ Connected to Kafka. Listening and writing JSON to S3...

🪙 BTC | $101514.0
🪙 USDT | $0.9999
🪙 XRP | $2.3693323254
🪙 BNB | $952.75
🪙 SOL | $152.89
🪙 USDC | $1.0
🪙 STETH | $3396.31
🪙 TRX | $0.29513
🪙 DOGE | $0.16969
🪙 ADA | $0.5464081428
🪙 WSTETH | $4136.438426332001
🪙 WBTC | $101442.75033299999
🪙 HYPE | $38.235
🪙 WETH | $3408.04132541529
🪙 META | $6.2735
🪙 LINK | $15.166163214
🪙 BCH | $504.11758046
🪙 XLM | $0.2784
🪙 WEETH | $3666.2442649416703
🪙 ZEC | $530.76
🪙 LEO | $9.123
🪙 USDE | $0.9992
🪙 LTC | $97.4532576
🪙 HBAR | $0.17457
🪙 SUI | $1.97
🪙 XMR | $391.6
🪙 AVAX | $16.83
🪙 BTCB | $101371.97570072615
🪙 SHIB | $9.52e-06
🪙 DAI | $0.99999
🪙 TON | $2.0003
🪙 UNI | $7.499
🪙 CRO | $0.12414
🪙 MNT | $1.2686
🪙 TAO | $350.6
🪙 WLFI | $0.1416
🪙 ICP | $6.31
🪙 PYUSD | $0.9992
🪙 NEAR | $2.555
🪙 AAVE | $204.86
🪙 BGB | $4.0495816
🪙 USD1 | $0.9992
🪙 OKB | $123.33
🪙 M | $2.439909
🪙 PEPE | $5.726e-06
🪙 ETC | $15.31
🪙 ENA | $0.3069430679380938
🪙 ASTER | $1.1193
🪙 APT | $3.0555
🪙 JLP | $4.9644
🪙 ONDO | $0.6

KeyboardInterrupt: 